In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn


In [282]:

okuzeni_all = pd.read_csv('okuzeni.csv')
NAMES = [ "ljubljana", "maribor", "kranj", "koper", "celje", "novo_mesto", "velenje", "nova_gorica", "krško", "ptuj", "murska_sobota", "slovenj_gradec"]
NAMES = 'ljubljana'

okuzeni = torch.tensor(okuzeni_all[NAMES].values).float()
okuzeni_diff = okuzeni.diff(axis=0)
okuzeni_log = torch.log(okuzeni)

scaler = MinMaxScaler()
okuzeni_norm = scaler.fit_transform(okuzeni.reshape(-1, 1))


In [289]:
train_data = okuzeni_norm

def create_sequences(data, seq_length, pred_length):
    """ Sprejme 
            data ... torch tensor 
            seq_length ... dolzina sekvenc
        Vrne 
    """
    sequences = []
    for i in range(len(data)-seq_length-pred_length):
        sequences.append((data[i:i+seq_length], data[i+seq_length:i+seq_length+pred_length]))
    return sequences

def create_sequences_normal(data, seq_length, pred_length):
    """ Sprejme 
            data ... torch tensor 
            seq_length ... dolzina sekvenc
        Vrne 
    """
    sequences = []
    m_s = []
    for i in range(len(data)-seq_length-pred_length):
        seq_learn = data[i:i+seq_length]
        m = torch.mean(seq_learn)
        s = torch.std(seq_learn)
        seq_learn = (seq_learn-m)/s
        seq_pred = data[i+seq_length:i+seq_length+pred_length]
        seq_pred = (seq_pred-m)/s

        sequences.append((seq_learn, seq_pred))
        m_s.append((s,m))
    return sequences, m_s


train_data_diff = okuzeni_diff
train_data_log = okuzeni_log

seq_length = 20
pred_length = 7

train_sequences = create_sequences(train_data, seq_length, pred_length)
#train_sequences, ms = create_sequences_normal(train_data, seq_length, pred_length)

In [291]:
class myLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(myLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.tanh = nn.Tanh()
        self.sig = nn.Sigmoid()

        self.forget_gate = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.input_gate = nn.Linear(input_size + hidden_size, hidden_size)
        self.exit_gate = nn.Linear(input_size + hidden_size, hidden_size)

        self.hidden = torch.zeros(hidden_size)
        self.cell = torch.zeros(hidden_size)

    def forward(self, x):
        combined = torch.cat((x, self.hidden))

        ft = self.sig(self.forget_gate(combined))
        it = self.sig(self.input_gate(combined))
        ct_hat = self.tanh(self.i2h(combined))
        ct = ft*self.cell + it*ct_hat
        ot = self.sig(self.exit_gate(combined))
        ht = ot * self.tanh(ct)

        self.cell = ct.detach()
        self.hidden = ht.detach()

        return ht
    
    def reset_hidden(self):
        self.hidden = torch.zeros(self.hidden_size)
        self.cell = torch.zeros(self.hidden_size)

class myLSTMnet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(myLSTMnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.gru_cell = myLSTMCell(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def reset_hidden(self):
        self.gru_cell.reset_hidden()

    def forward(self, x):
        output = []
        for t in range(x.shape[0]):
            h = self.gru_cell(x[t])
            output += [self.output_layer(h)]

        output = torch.stack(output)
        return output[-1]

In [ ]:
model = myLSTMnet(1, 32, 1)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 1
for epoch in range(epochs):
    print("Epoch ", epoch)
    for i, (seq, labels) in enumerate(train_sequences):
        optimizer.zero_grad()
        model.reset_hidden()
        y_pred = model(seq)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        if i%100 == 0:
            print(f'i: {i:03d}, Loss: {loss:.4f}')

In [ ]:
test_inputs = train_data[-seq_length:].tolist()

model = model.eval()
test_size = train_data.shape[0]-seq_length

pred = []
true = []
for i in range(len(train_sequences)):
    with torch.no_grad():
        model.reset_hidden()
        seq = train_sequences[i][0]
        res = model(seq)
        #print(res)
        #print(train_sequences[i][1])
        pred.append(res[0].item())
        true.append(train_sequences[i][1][0])
#        
#for i in range(test_size):
#    seq = torch.FloatTensor(test_inputs[-seq_length:])
#    with torch.no_grad():
#        model.reset_hidden()
#        res = model(seq)
#        test_inputs.append(res[0].item())
from sklearn.metrics import r2_score
#test_data = train_data[seq_length:-pred_length]

#print(test_data.shape)
#print(len(test_inputs[seq_length:-pred_length]))
#print(r2_score(test_data.numpy(), test_inputs[seq_length:-pred_length]))
print(r2_score(true, pred))


plt.plot(pred)
plt.show()
plt.plot(true)
#plt.plot(okuzeni_log)
plt.show()
#actual_predictions = test_inputs[seq_length:]
#plt.plot(okuzeni_diff.data.numpy()[-test_size:])
#plt.plot(actual_predictions)
#plt.show()

In [103]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.rnn(x.view(len(x), 1, -1))
        x = self.fc(x.view(len(x), -1))
        return x[-1]



model = RNN(1, 32, 7)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
epochs = 10

for epoch in range(epochs):
    for i, (seq, labels) in enumerate(train_sequences):
        optimizer.zero_grad()
        model.hidden = (torch.zeros(1,1,model.hidden_size))
        y_pred = model(seq)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        #if i%100 == 0:
        #    print(f'i: {i:03d}, Loss: {loss:.4f}')


In [ ]:

test_inputs = train_data[-seq_length:].tolist()

model = model.eval()
test_size = train_data.shape[0]-seq_length-pred_length
print(test_size)
for i in range(test_size-1):
    seq = torch.FloatTensor(test_inputs[-seq_length:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1,1,model.hidden_size))
        res = model(seq)
        print(res)
        print(train_sequences[i][1])
        test_inputs.append(res[0].item())

from sklearn.metrics import r2_score
test_data = train_data[seq_length:-pred_length]

print(test_data.shape)
print(len(test_inputs[seq_length:-pred_length]))
print(r2_score(test_data.numpy(), test_inputs[seq_length:-pred_length]))


plt.plot(test_inputs[seq_length:-pred_length])
plt.plot(train_data_diff)
#plt.plot(okuzeni_log)
plt.show()
